# MDS

### Contents
1. Formulation (For Managers)
2. Practical implementation (For Data Engineers)
3. Proof & Maths (For Data Scientists)


### Introduction

Multidimensional scaling is a concept that appeared in statistics in 1950s. It's goal is to find such a projection onto some lower-dimensional space so that it <u>preserves all distances</u> between the points as much as possible. Most frequently MDS is used for data visualization, so the new dimensionality is often chosen to be 2 or 3.

\# Most known with contributions by Kruskal.

A couple of notation notes:
- in MDS the target space is often referred to as [ordination](https://en.wikipedia.org/wiki/Ordination_(statistics)) space
     because in non-metric MDS what we do is we try to ordinate points in order to retain the structure of a dataset
- dataset coordinates in a target space is called configuration
     because dataset coordinates is exactly what is being optimized

### Problem statement

What we've got as an input?

Unlike PCA, we don't have dataset coordinates, we have to deal some [dissimilarity](https://en.wikipedia.org/wiki/Distance_matrix) matrix - a square matrix that contains pairwise distances between the points.

Whether in PCA we are rtying to retain high-level axis-based linear structure, here the structure that we are trying to preserve is way more detailed and complex.

So let's denote our input dissimilarity matrix as D_{old}

$$D^{old}_{ij} = dis(x_i,x_j)$$

Suppose we did some mapping and now have 2 dissimilarity matrixes $D_{old}$ in the original space and $D_{new}$ in the target space. The goal is to preserve distances as much as possible.

That means:
- we need to define some loss function that compares old and new distance matrices $L(D_{old}, D_{new})$
- we need to find a configuration $X_{new}$ that<u>minimizes</u> the loss function L
So, to this point, we formulated some optimization problem. 

#### Assumption 1
We will think of ${D_{new}}$ as a matrix of <u>Euclidean</u> distances.

It does not make much sense to choose more complex distances, since we need it only for visualiztion:

$$D^{new}_{ij}(X) = {||x_i - x_j ||}^2 $$



#### Assumption 2

In general $D_{old}$ can contain any distance metrics (depending on the alogirithm variant).

But in case of classic metric-MDS we also assume that $D_{old}$ was <u>also</u> calculated using Euclidean distance:

(though in general, there are MDS variants that allow other metrics)

$$D^{old}_{ij}(X) = {||x_i - x_j ||}^2 $$

$$X_{new} = \underset{X}{\operatorname{{arg\,min}}} \sum_{i \ne j} {\left( D^{new}_{ij}(X) - D^{old}_{ij}\right)}^2$$

The scheme of the process is described below:

<img src = "img/mds.png" width=500>

- We don't observe original coordinates.
- We need to find X that minimaizes.

### So how we do it for classic MDS?
1. We compute a decentered distance matrix. 
    
        Each element represents distances between a pair of points in a dataset
2. We do eigendecomposition of that distance matrix 
        
        It gives us a set of eigenvectors of that matrix
3. We project original dataset onto the first K components

That's it.

In a couple of minutes I will give a more formal proof.

#### Relation to PCA

MDS is similar to PCA in a way that it also uses the eigendecomposition to get reduced coordinates. It's even refered to as PCoA (Principal Coordinate Analysis).

But the formulation is a bit different
- for PCA = to preserve data variation after mapping
- for MDS = to preserve pairwise distances after mapping

Also in MDS we work with distance matrix instead of covariance matrix, which makes this approach a bit more general, becasuse we can apply arbitrary metrics.

####  Flavours of MDS

As usual, there are several enhancements of classic MDS:
- two popular ones are metric-MDS and non-metruc MDS



### Proof outline

1. We'll describe our algorithm in matrix notation
2. We'll show that eigenvectors projection gives optimal approximation

### Proof

#### Step 1

As a quick aside, let's explore what the centering matrix is.

[Centering matrix](https://en.wikipedia.org/wiki/Centering_matrix) is a concise way to subtract mean from matrix

If we denote $I_n$ an identity matrix; $J_n$ - matrix of all ones, then we can formulate it as:

$$C_n =  I_n - \tfrac{1}{n}J_n$$

    For example:

$$C_3 = \left[ \begin{array}{rrr}
1 & 0 & 0 \\
0 & 1 & 0 \\
0 & 0 & 1 
\end{array} \right] -  \frac{1}{3}\left[ \begin{array}{rrr}
1 & 1 & 1 \\
1 & 1 & 1 \\
1 & 1 & 1 
\end{array} \right]
 = \left[ \begin{array}{rrr}
\frac{2}{3} & -\frac{1}{3} & -\frac{1}{3} \\
-\frac{1}{3} & \frac{2}{3} & -\frac{1}{3} \\
-\frac{1}{3} & -\frac{1}{3} & \frac{2}{3} 
\end{array} \right]$$


Right multiplication <u>subtracts row mean</u> from each row 
$$XC$$

Left multiplication <u>subtracts column mean</u> from each column
$$CX$$

Double centering = applying demeaning matrices twice
$$CXC$$

In that case each row and column will have an average of zero.

#### Step 2
Let's describe $D_{old}$ in original (unknown) coordinates:

$$d_{ij}={\left\lVert x_i - x_j \right\rVert}^2 = (x_i-x_j, x_i-x_j)$$ 

$$d_{ij}= {\left\lVert x_i \right\rVert}^2 -2\left\lVert x_i\right\rVert \left\lVert x_j \right\rVert + {\left\lVert x_j\right\rVert}^2$$

In a matrix notation:

$$D^X =  Z - 2X^TX + Z^T$$

where $Z$ and $Z^T$ contain squared norms of two points; $X^TX$ are dot products of point pairs; 

<img src="img/mds_proof1.png" width=700>



Those products in the middle are used to subtract part of distance explained by two points similarity. If points are perpendicular, the distance between them is just a sum of squares.

-----

#### Step 3

Let's apply double decentering to our distance matrix $D^X$

$Z$ and $Z^T$ are row/column constant => they vanish out

So we get:

$$CD^XC = -2CX^TXC$$

Now we can rewrite it a bit:

$$CX^TXC = (CX^T)(XC) = (XC^T)^T(XC) =  (XC)^T(XC) = \hat{X}^T\hat{X}$$

Explaining each equation:
- matrix multiplication is associative
- $(AB)^T = B^TA^T$
- $C^T=C$ because C is symmetric

Here $\hat{X}^T$ and $\hat{X}$ - decentered coordinate matrices.

Actually that's quite an obvious fact - distance matrix don't care about absolute coordinates => it correspond to decentered dataset



Let's denote this decentered distance matrix by $B^X$:

$$B^X = -\frac{1}{2}CD^XC = \hat{X}^T\hat{X}$$

#### Step 4

Now we need to find best rank-k approximation of decentered distance matrix $\hat{X}^T\hat{X}$. 

$$B^y = \underset{B^Y}{argmin} = ||B^X - B^Y||^2 = ||\hat{X}^T\hat{X} - Y^TY||^2$$

Notice that Y's that we are seeking for, will also represent the decentered coordinates, not the original ones

Notice that here we use Frobenius norm - it is the same as minimizaing mean squared error.

-----

#### Step 5

Such optimization is done by SVD decomposition. 

There is a theorem. Suppose we deal with matrix A, then
<img src="img/svd_theorem.png" width=300>


Those Y optimally approximate B^X:
$$Y^TY = (UD^{1/2})(D^{1/2}U)$$



### References

[SVD](https://www.cs.cmu.edu/~venkatg/teaching/CStheory-infoage/book-chapter-4.pdf)





Stress

Scree plot

Shepard diagram

# 2. Practical implementation

In [31]:
from sklearn.manifold import MDS

mds = MDS(

    # target dimensionality
    n_components=2,
    
    # whether to perform metric or non-metric MDS
    metric=True,
    
    # whether to use dissimilarity matrix as an input (precomputed) or it's required to compute them (euclidean)
    dissimilarity='euclidean',
    
    # minimal change in Stress criterion to continue
    eps=0.001,
    
    # number of algorithm runs
    n_init=4, 
    
    # iterations limit
    max_iter=300, 
    
    # logging verbosity
    verbose=2, 
        
    # number of parallel execution threads
    n_jobs=None, 
    
    # random seed for reproducibility
    random_state=None,
    
)

# Load some Data
from sklearn.datasets import load_digits
X, _ = load_digits(return_X_y=True)

mds.fit(X)



it: 0, stress 3799400497.3742704
it: 1, stress 770435675.8429354
it: 2, stress 752914366.0691837
it: 3, stress 740497870.0252047
it: 4, stress 730867323.9820273
it: 5, stress 723218782.083574
it: 6, stress 717005897.8040385
it: 7, stress 711842699.2150128
it: 8, stress 707434584.1541102
it: 9, stress 703607200.9586215
it: 10, stress 700288427.4172852
it: 11, stress 697401838.3374339
it: 12, stress 694873231.604623
it: 13, stress 692642725.94605
it: 14, stress 690685586.7856764
it: 15, stress 688972230.9229314
it: 16, stress 687468707.9364468
it: 17, stress 686155968.2967236
it: 18, stress 684995498.3255575
it: 19, stress 683945444.6885906
it: 20, stress 682980855.4721786
it: 21, stress 682076989.6825824
it: 22, stress 681216066.4441749
it: 23, stress 680392158.8859686
it: 24, stress 679597023.6133329
it: 25, stress 678820268.8768164
it: 26, stress 678053039.4305738
it: 27, stress 677285907.3994784
it: 28, stress 676506429.1618745
it: 29, stress 675699188.0037136
it: 30, stress 67485386

it: 244, stress 420791364.266147
it: 245, stress 420749223.225202
it: 246, stress 420708809.3110478
it: 247, stress 420670205.5683677
it: 248, stress 420633198.5950358
it: 249, stress 420598176.69052017
it: 250, stress 420565263.615261
it: 251, stress 420534311.59111506
it: 252, stress 420505130.8675949
it: 253, stress 420476973.95458406
it: 254, stress 420449711.4817981
it: 255, stress 420423430.1635868
it: 256, stress 420397787.85471934
it: 257, stress 420372451.5921675
it: 258, stress 420347808.2159612
it: 259, stress 420323120.73675257
it: 260, stress 420298391.86208576
it: 261, stress 420273794.7207847
it: 262, stress 420248768.46011436
it: 263, stress 420223301.0748619
it: 264, stress 420197148.814359
it: 265, stress 420170478.7163914
it: 266, stress 420143490.50070995
it: 267, stress 420116314.698665
it: 268, stress 420089828.72637236
it: 269, stress 420064384.0719554
it: 270, stress 420041061.86392397
it: 271, stress 420019612.2241951
it: 272, stress 420000384.2333597
it: 273, 

it: 187, stress 433512095.54955244
it: 188, stress 433444843.2703401
it: 189, stress 433379824.26228404
it: 190, stress 433316922.7596936
it: 191, stress 433256257.6333381
it: 192, stress 433196449.0100839
it: 193, stress 433137419.33661354
it: 194, stress 433079368.15494627
it: 195, stress 433021968.3328179
it: 196, stress 432965185.8321788
it: 197, stress 432908738.1470869
it: 198, stress 432852704.68612385
it: 199, stress 432797000.3296088
it: 200, stress 432740808.67975885
it: 201, stress 432684453.2954383
it: 202, stress 432628177.5314059
it: 203, stress 432570325.69110924
it: 204, stress 432511034.73898464
it: 205, stress 432449944.1800868
it: 206, stress 432386286.2620841
it: 207, stress 432321409.02512324
it: 208, stress 432254739.37601465
it: 209, stress 432186490.68339616
it: 210, stress 432117943.15836495
it: 211, stress 432052104.7574251
it: 212, stress 431989752.8625228
it: 213, stress 431929320.24356925
it: 214, stress 431871005.8196425
it: 215, stress 431813072.24490744


it: 131, stress 432161029.3574795
it: 132, stress 431742561.5851235
it: 133, stress 431344505.036208
it: 134, stress 430964799.15522647
it: 135, stress 430598812.17478085
it: 136, stress 430247067.1348511
it: 137, stress 429909103.29275346
it: 138, stress 429585493.6705122
it: 139, stress 429275699.0275685
it: 140, stress 428979443.6344899
it: 141, stress 428694018.16604626
it: 142, stress 428418474.4217912
it: 143, stress 428153653.073741
it: 144, stress 427902661.8368451
it: 145, stress 427664656.47048944
it: 146, stress 427439473.2472276
it: 147, stress 427222882.2402743
it: 148, stress 427009929.2467322
it: 149, stress 426801341.33473504
it: 150, stress 426597246.7919388
it: 151, stress 426397434.74572206
it: 152, stress 426202238.68757176
it: 153, stress 426012700.40441203
it: 154, stress 425830357.634249
it: 155, stress 425655374.6811689
it: 156, stress 425487965.69260544
it: 157, stress 425325739.06929
it: 158, stress 425169057.4063267
it: 159, stress 425015129.97270805
it: 160,

it: 74, stress 526234980.794612
it: 75, stress 522528605.7202079
it: 76, stress 518877145.82227856
it: 77, stress 515269357.8740802
it: 78, stress 511728118.0392189
it: 79, stress 508285591.2553429
it: 80, stress 504955474.66648203
it: 81, stress 501741541.77139294
it: 82, stress 498663178.1654813
it: 83, stress 495688974.2800723
it: 84, stress 492818998.2032675
it: 85, stress 490057340.32508326
it: 86, stress 487400967.54419106
it: 87, stress 484846862.8447412
it: 88, stress 482378319.98380435
it: 89, stress 479988566.16746515
it: 90, stress 477689232.0303662
it: 91, stress 475475236.519892
it: 92, stress 473362928.00163937
it: 93, stress 471336021.6891541
it: 94, stress 469394174.5006728
it: 95, stress 467517932.3704737
it: 96, stress 465688518.83927476
it: 97, stress 463910888.4479034
it: 98, stress 462163652.15062195
it: 99, stress 460450522.7083028
it: 100, stress 458772664.8623975
it: 101, stress 457149686.18902034
it: 102, stress 455588354.7911961
it: 103, stress 454081078.75409

MDS(verbose=2)

Fit results

In [28]:
#ndarray of shape (n_samples, n_components). Stores the position of the dataset in the embedding space.
print(f"2D point embeddings = {mds.embedding_}\n")

# The final value of the stress (sum of squared distance of the disparities and the distances for all constrained points).
print(f"Minimum Stress value = {mds.stress_}\n")

#_ndarray of shape (n_samples, n_samples) Pairwise dissimilarities between the points. Symmetric matrix that: either uses a custom dissimilarity matrix by setting dissimilarity to ‘precomputed’; or constructs a dissimilarity matrix from data using Euclidean distances.
print(f"Dissimilairy Matrix = {mds.dissimilarity_matrix_}\n")

# The number of iterations corresponding to the best stress
print(f"Number of iterations = {mds.n_iter_}\n")


2D point embeddings = [[ 28.91982813  -3.64270726]
 [-22.8755649   20.91498596]
 [-18.14213693  28.08614231]
 ...
 [ -5.01392435  17.95685277]
 [ 18.38084092  -7.04435349]
 [  8.97189495   1.33453808]]

Minimum Stress value = 425727713.77070963

Dissimilairy Matrix = [[ 0.         59.55669568 54.12947441 ... 50.37856687 37.06750599
  47.03190407]
 [59.55669568  0.         41.62931659 ... 38.58756276 48.56953778
  50.32891813]
 [54.12947441 41.62931659  0.         ... 38.34057903 50.7740091
  43.95452195]
 ...
 [50.37856687 38.58756276 38.34057903 ...  0.         44.15880433
  28.87905816]
 [37.06750599 48.56953778 50.7740091  ... 44.15880433  0.
  39.42080669]
 [47.03190407 50.32891813 43.95452195 ... 28.87905816 39.42080669
   0.        ]]

Number of iterations = 300



In [6]:
import sklearn